# Fine-Tuning de Qwen 2.5 para Análisis de Logs (vía Unsloth)

Este notebook permite entrenar el modelo Qwen 2.5 utilizando el dataset de logs preparado. Está optimizado para ejecutarse en **Google Colab** con una GPU (T4, L4 o A100).

## 1. Instalación de Dependencias

Utilizamos **Unsloth** para un entrenamiento 2x más rápido y con un 70% menos de memoria.

In [1]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Colab tiene torch 2.4 / 2.5, así que necesitamos una instalación específica
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.29" "trl<0.13.0" peft accelerate bitsandbytes
!pip install datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-tzgvtomd/unsloth_ca2ae92b186341b9848cc49c98887ed2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-tzgvtomd/unsloth_ca2ae92b186341b9848cc49c98887ed2
  Resolved https://github.com/unslothai/unsloth.git to commit e51d3ea2e498fc893770d92ca6727bd113918480
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 31.0 MB/s eta 0:00:00
 

## 2. Clonar el Repositorio

Como el repositorio es privado, necesitamos autenticarnos. Tienes dos opciones:
1. Autorizar Google Colab en la organización `kaizen-org`.
2. Usar un **Personal Access Token (PAT)** de GitHub.

In [2]:
import os
from getpass import getpass
from google.colab import userdata

REPO_URL = "github.com/kaizen-org/qwen-destilled.git"

if not os.path.exists("qwen-destilled"):
    token = userdata.get('PATgithub')
    !git clone https://{token}@{REPO_URL}
    %cd qwen-destilled
else:
    %cd qwen-destilled
    !git pull

Introduce tu GitHub Personal Access Token (PAT): ··········
Cloning into 'qwen-destilled'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 29), reused 48 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 58.67 KiB | 11.73 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/qwen-destilled


## 3. Configuración y Entrenamiento

Cargamos el modelo Qwen 2.5 (7B en 4 bits por defecto) y lanzamos el entrenamiento con el archivo `train_dataset.jsonl`.

In [4]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# --- CONFIGURACIÓN ---
MAX_SEQ_LENGTH = 1024 # Reducir para ahorrar memoria
MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
DATASET_FILE = "train_dataset.jsonl"
OUTPUT_DIR = "qwen_log_analyzer"

# --- 1. CARGAR MODELO ---
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
    device_map="auto", # Añadir para asegurar la correcta asignación del dispositivo
)

# --- 2. CONFIGURAR LoRA ---
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

# --- 3. PREPARAR DATASET ---
def formatting_prompts_func(examples):
    texts = []
    for input_text, output_text in zip(examples['input'], examples['output']):
        messages = [
            {"role": "system", "content": "Eres un experto en SRE y DevOps especializado en análisis de logs."},
            {"role": "user", "content": f"Analiza este patrón de log y proporciona diagnóstico y acción recomendada:\n\n{input_text}"},
            {"role": "assistant", "content": output_text}
        ]
        texts.append(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False))
    return {"text": texts}

dataset = load_dataset("json", data_files=DATASET_FILE, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# --- 4. TRAINER ---
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    args=TrainingArguments(
        per_device_train_batch_size=1, # Reducir para ahorrar memoria
        gradient_accumulation_steps=8, # Aumentar para mantener el tamaño de batch efectivo
        warmup_steps=5,
        max_steps=60, # Ajusta según el tamaño de tu dataset
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

trainer.train()

==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/106 [00:00<?, ? examples/s]

/content/qwen-destilled/unsloth_compiled_cache/UnslothSFTTrainer.py:875: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/content/qwen-destilled/unsloth_compiled_cache/UnslothSFTTrainer.py:903: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/106 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


TypeError: device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index = -1)


## 4. Prueba de Inferencia

Probamos el modelo recién entrenado con un log de error.

In [ ]:
FastLanguageModel.for_inference(model)

test_log = "[ERROR] 504 Gateway Timeout while calling product-service at /v1/prices"
messages = [
    {"role": "system", "content": "Eres un experto en SRE y DevOps especializado en análisis de logs."},
    {"role": "user", "content": f"Analiza este patrón de log y proporciona diagnóstico y acción recomendada:\n\n{test_log}"}
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))